### Transformer encoder
- 사실 transformer encoder구조는 매우 단순하다.
  - 그냥 transformer layer를 여러개 쌓은거다.
  
### transformer encoder layer
- 단순하게 보면 SA(self attention) block, FF(feed forward) block두개로 이루어져있다.
  - 맘편하게 두개 구현을 따로 해놓으면편하다.
  
### FF block
- 쉬운거부터 끝내자
  - x += self.dropout2(self.linear2(self.dropout(self.activation(self.linear1(x)))))
    - linear로 hidden dim만큼 확장했다가 relu&dropout하고 다시 원상복귀 시키고 dropout만 먹인다. 그리고 원래의 x랑 더해준다
    
### SA block
- 단순하게 접근하자면 self attention한 layer를 원래의 layer에 더해주는 거다.
  - x = x + self._sa_block(x)
- se_block은 대충(x, x, x, src_mask, key_padding_mask)이렇게 입력받는 multi head attention을 쓴다. 참고로 nhead라고 사전에 정의해줘야한다.
  - MultiheadAttention 요거가 사실상 transformer가 더럽게 만드는 거의 모든것..
    - 얼마나 징그럽냐면 forward 함수를 functional에서 따로 정의할정도
    - multi_head_attention_forward 이거는 이따 공부하고 일단 필요한 원소를 정리하는 layer 정의 단계부터 공부해본다
  - 근본은 q k v 쿼리 키 벨류인데, 이건그냥 dim 맞춰서 linear하게 생성된거라 생각하면 맘편하다.
    - 처음 정의할때 넣어주는게 
      - ninp, nhead, nhid, dropout 이걸 넣어주는데 의미는 아래랑 같다. 간접적으로 이해해보자
        - ninp = emsize = 200 # 임베딩 차원
        - nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
        - nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수 
          - 참고로 ninp는 nhead로 나누어 떨어져야 ㅎ나다.
        - dropout = 0.2 # 드랍아웃(dropout) 값
  - 중간에 생성되는 변수들을 살펴보자
    - kdim, vdim  요거는 중간에 사용될 변수인데 이들이 최종 입력이자 출력인 embed_dim(ninp)과 같은지가 중요하다.
      - xavier_uiniform_을 쓰는데 이게 파라미터 숫자가중요하기 때문
    - 다같으면 파라미터를 한번에 초기화하고 같지 않으면 다 따로 초기화 한다.
      - self.v_proj_weight = Parameter(torch.empty((embed_dim, self.vdim), **factory_kwargs))
        - 요런 느낌으로 변환 함수도 필요하다.. 진짜 귀찮게 생겼다.
        - 참고로 empty는 random noise가 들어간다 그런데 그 값 범위가 좀 매우 넓어서 그냥 쓰면 절대 안되고 초기화 꼭 해주자.
        - parameter는 grad를 갖는 tensor 변수로 정의하는거다.

In [7]:
import torch
a = torch.empty((3 * 4, 4))
b = torch.nn.Parameter(torch.empty((3 * 4, 4)))

        

In [17]:
print(a)
print(b)
print(b.chunk(2, 0))
print(b.chunk(2, 1))

tensor([[ 2.8026e-45,  5.6052e-45,  2.8026e-45,  3.7835e-44],
        [ 2.8026e-45,  0.0000e+00,  2.8026e-45,  1.4013e-45],
        [ 3.3631e-44,  3.0707e-41,  1.0678e+37,  4.5597e-41],
        [ 1.4013e-45,  0.0000e+00, -2.3686e-34,  4.5597e-41],
        [ 2.8026e-45,  5.6052e-45,  2.8026e-45,  1.2612e-44],
        [ 2.9427e-44,  3.7835e-44,  2.6275e+04,  3.0707e-41],
        [-1.5583e-33,  4.5597e-41,  1.4013e-45,         nan],
        [ 2.8026e-45,  5.6052e-45,  2.8026e-45,  2.1019e-44],
        [ 1.4013e-45,  0.0000e+00,  2.6276e+04,  3.0707e-41],
        [ 2.8026e-45,  0.0000e+00,  2.6275e+04,  3.0707e-41],
        [ 2.6275e+04,  3.0707e-41,  2.8026e-44,  3.0707e-41],
        [ 2.7397e+01,  3.0707e-41,  0.0000e+00,  0.0000e+00]])
Parameter containing:
tensor([[ 2.8026e-45,  3.0707e-41,  0.0000e+00,  0.0000e+00],
        [ 4.2039e-45,  0.0000e+00,  4.2039e-45,  1.4013e-45],
        [ 3.3631e-44,  3.5032e-44,  1.0678e+37,  4.5597e-41],
        [ 1.4013e-45,  4.5597e-41,  1.4013e-45,

In [4]:
1.8376e+14

183760000000000.0

In [48]:
class Transformer(nn.Module):

	def __init__(self, src_embed, trg_embed, encoder, decoder, fc_layer):
		super(Transformer, self).__init__()
		self.src_embed = src_embed
		self.trg_embed = trg_embed
		self.encoder = encoder
		self.decoder = decoder
		self.fc_layer = fc_layer

	def forward(self, src, trg, src_mask, trg_mask):
		encoder_output = self.encoder(self.src_embed(src), src_mask)
		out = self.decoder(self.trg_embed(trg), trg_mask, encoder_output, src_mask)
		out = self.fc_layer(out)
		out = F.log_softmax(out, dim=-1)
		return out

In [25]:
class Encoder(nn.Module):

	def __init__(self, encoder_layer, n_layer):  # n_layer: Encoder Layer의 개수
		super(Encoder, self).__init__()
		self.layers = []
		for i in range(n_layer):
			self.layers.append(copy.deepcopy(encoder_layer))

	def forward(self, x, mask):
		out = x
		for layer in self.layers:
			out = layer(out, mask)
		return out

In [26]:
class EncoderLayer(nn.Module):

	def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
		super(EncoderLayer, self).__init__()
		self.multi_head_attention_layer = multi_head_attention_layer
		self.position_wise_feed_forward_layer = position_wise_feed_forward_layer

	def forward(self, x, mask):
		out = self.multi_head_attention_layer(query=x, key=x, value=x, mask=mask)
		out = self.position_wise_feed_forward_layer(out)
		return out

In [18]:
def calculate_attention(self, query, key, value, mask):
	# query, key, value's shape: (n_batch, seq_len, d_k)
	d_k = key.size(-1) # get d_k
	attention_score = torch.matmul(query, key.transpose(-2, -1)) # Q x K^T, attention_score's shape: (n_batch, seq_len, seq_len)
	attention_score = attention_score / math.sqrt(d_k) # scaling
	if mask is not None:
		attention_score = score.masked_fill(mask==0, -1e9) # masking
	attention_prob = F.softmax(score, dim=-1) # softmax, attention_prob's shape: (n_batch, seq_len, seq_len)
	out = torch.matmul(attention_prob, value) # Attention_Prob x V, out's shape: (n_batch, seq_len, d_k)
	return out

In [23]:
import copy

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, d_model, h, qkv_fc_layer, fc_layer):
        # qkv_fc_layer's shape: (d_embed, d_model)
        # fc_layer's shape: (d_model, d_embed)
        super(MultiHeadAttentionLayer, self).__init__()
        self.d_model = d_model
        self.h = h
        self.query_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.key_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.value_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.fc_layer = fc_layer
        
    def forward(self, query, key, value, mask=None):
        # query, key, value's shape: (n_batch, seq_len, d_embed)
        # mask's shape: (n_batch, seq_len, seq_len)
        n_batch = query.shape[0] # get n_batch

        def transform(x, fc_layer): # reshape (n_batch, seq_len, d_embed) to (n_batch, h, seq_len, d_k)
            out = fc_layer(x) # out's shape: (n_batch, seq_len, d_model)
            out = out.view(n_batch, -1, self.h, self.d_model//self.h) # out's shape: (n_batch, seq_len, h, d_k)
            out = out.transpose(1, 2) # out's shape: (n_batch, h, seq_len, d_k)
            return out

        query = transform(query, self.query_fc_layer) # query, key, value's shape: (n_batch, h, seq_len ,d_k)
        key = transform(key, self.key_fc_layer)
        value = transform(value, self.value_fc_layer)

        if mask is not None:
            mask = mask.unsqueeze(1) # mask's shape: (n_batch, 1, seq_len, seq_len)

        out = self.calculate_attention(query, key, value, mask) # out's shape: (n_batch, h, seq_len, d_k)
        out = out.transpose(1, 2) # out's shape: (n_batch, seq_len, h, d_k)
        out = contiguous().view(n_batch, -1, self.d_model) # out's shape: (n_batch, seq_len, d_model)
        out = self.fc_layer(out) # out's shape: (n_batch, seq_len, d_embed)
        return out
        
        
        

In [27]:
class PositionWiseFeedForwardLayer(nn.Module):
	def __init__(self, first_fc_layer, second_fc_layer):
		self.first_fc_layer = first_fc_layer
		self.second_fc_layer = second_fc_layer
	
	def forward(self, x):
		out = self.first_fc_layer(x)
		out = F.relu(out)
		out = self.dropout(out)
		out = self.second_fc_layer(out)
		return out

In [28]:
class ResidualConnectionLayer(nn.Module):
	def __init__(self, norm_layer):
		super(ResidualConnectionLayer, self).__init__()
		self.norm_layer = norm_layer

	def forward(self, x, sub_layer):
		out = sub_layer(x) + x
		out = self.norm_layer(out)
		return out

In [29]:
class EncoderLayer(nn.Module):

	def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
		super(EncoderLayer, self).__init__()
		self.multi_head_attention_layer = multi_head_attention_layer
		self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
		self.residual_connection_layers = [ResidualConnectionLayer(copy.deepcopy(norm_layer)) for i in range(2)]

	def forward(self, x, mask):
		out = self.residual_connection_layers[0](x, lambda x: self.multi_head_attention_layer(x, x, x, mask))
		out = self.residual_connection_layers[1](x, lambda x: self.position_wise_feed_forward_layer(x))
		return out

In [39]:
import numpy as np
def subsequent_mask(size):
    atten_shape = (1, size, size)
    mask = np.triu(np.ones(atten_shape), k=1).astype('uint8') # masking with upper triangle matrix
#     print(mask)
    out = torch.from_numpy(mask)==0 # reverse (masking=False, non-masking=True)
#     print(out)
    return out


In [41]:
def make_std_mask(tgt, pad):
    tgt_mask = (tgt != pad) # pad masking
    tgt_mask = tgt_mask.unsqueeze(-2) # reshape (n_batch, seq_len) -> (n_batch, 1, seq_len)
    tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data)) # pad_masking & subsequent_masking
    return tgt_mask

In [43]:
class Decoder(nn.Module):
	def __init__(self, sub_layer, n_layer):
		super(Decoder, self).__init__()
		self.layers = []
		for i in range(n_layer):
			self.layers.append(copy.deepcopy(sub_layer))

	def forward(self, x, mask, encoder_output, encoder_mask):
		out = x
		for layer in self.layers:
			out = layer(x, mask, encoder_output, encoder_mask)
		return out

In [44]:
class DecoderLayer(nn.Module):
	def __init__(self, masked_multi_head_attention_layer, multi_head_attention_layer, position_wise_feed_forward_layer, norm_layer):
		super(DecoderLayer, self).__init__()
		self.masked_multi_head_attention_layer = ResidualConnectionLayer(masked_multi_head_attention_layer, copy.deepcopy(norm_layer))
		self.multi_head_attention_layer = ResidualConnectionLayer(multi_head_attention_layer, copy.deepcopy(norm_layer))
		self.position_wise_feed_forward_layer = ResidualConnectionLayer(position_wise_feed_forward_layer, copy.deepcopy(norm_layer))

	def forward(self, x, mask, encoder_output, encoder_mask):
		out = self.masked_multi_head_attention_layer(query=x, key=x, value=x, mask=mask)
		out = self.multi_head_attention_layer(query=out, key=encoder_output, value=encoder_output, mask=encoder_mask)
		out = self.position_wise_feed_forward_layer(x=out)
		return out

In [45]:
class TransformerEmbedding(nn.Module):
	def __init__(self, embedding, positional_encoding):
		super(TransformerEmbedding, self).__init__()
		self.embedding = nn.Sequential(embedding, positional_encoding)

	def forward(self, x):
		out = self.embedding(x)
		return out

In [46]:
class Embedding(nn.Module):
	def __init__(self, d_embed, vocab):
		super(Embedding, self).__init__()
		self.embedding = nn.Embedding(len(vocab), d_embed)
		self.vocab = vocab
		self.d_embed = d_embed

	def forward(self, x):
		out = self.embedding(x) * math.sqrt(self.d_embed)
		return out

In [47]:
class PositionalEncoding(nn.Module):
	def __init__(self, d_embed, max_seq_len=5000):
		super(PositionalEncoding, self).__init__()
		encoding = torch.zeros(max_seq_len, d_embed)
		position = torch.arange(0, max_seq_len).unsqueeze(1)
		div_term = torch.exp(torch.arange(0, d_embed, 2) * -(math.log(10000.0) / d_embed))
		encoding[:, 0::2] = torch.sin(position * div_term)
		encoding[:, 1::2] = torch.cos(position * div_term)
		self.encoding = encoding
	
	def forward(self, x):
		out = x + Variable(self.encoding[:, :x.size(1)], requires_grad=False)
		out = self.dropout(out)
		return out

In [49]:
def make_model(
    src_vocab, 
    trg_vocab, 
    d_embed = 512, 
    n_layer = 6, 
    d_model = 512, 
    h = 8, 
    d_ff = 2048):

    cp = lambda x: copy.deepcopy(x)

    # multi_head_attention_layer 생성한 뒤 copy해 사용
    multi_head_attention_layer = MultiHeadAttentionLayer(
                                    d_model = d_model,
                                    h = h,
                                    qkv_fc_layer = nn.Linear(d_embed, d_model),
                                    fc_layer = nn.Linear(d_model, d_embed))

    # position_wise_feed_forward_layer 생성한 뒤 copy해 사용    
    position_wise_feed_forward_layer = PositionWiseFeedForwardLayer(
                                        first_fc_layer = nn.Linear(d_embed, d_ff),
                                        second_fc_layer = nn.Linear(d_ff, d_embed))
    
    # norm_layer 생성한 뒤 copy해 사용
    norm_layer = nn.LayerNorm(d_embed, eps=1e-6)

    # 실제 model 생성
    model = Transformer(
                src_embed = TransformerEmbedding(    # SRC embedding 생성
                                embedding = Embedding(
                                                d_embed = d_embed, 
                                                vocab = src_vocab), 
                                positional_encoding = PositionalEncoding(
                                                d_embed = d_embed)), 
	
                trg_embed = TransformerEmbedding(    # TRG embedding 생성
                                embedding = Embedding(
                                                d_embed = d_embed, 
                                                vocab = trg_vocab), 
                                positional_encoding = PositionalEncoding(
                                                d_embed = d_embed)),
                encoder = Encoder(                    # Encoder 생성
                                sub_layer = EncoderLayer(
                                                multi_head_attention_layer = cp(multi_head_attention_layer),
                                                position_wise_feed_forward_layer = cp(position_wise_feed_forward_layer),
                                                norm_layer = cp(norm_layer)),
                                n_layer = n_layer),
                decoder = Decoder(                    # Decoder 생성
                                sub_layer = DecoderLayer(
                                                masked_multi_head_attention_layer = cp(multi_head_attention_layer),
                                                multi_head_attention_layer = cp(multi_head_attention_layer),
                                                position_wise_feed_forward_layer = cp(position_wise_feed_forward_layer),
                                                norm_layer = cp(norm_layer)),
                                n_layer = n_layer),
                fc_layer = nn.Linear(d_model, len(trg_vocab)))    # Generator의 FC Layer 생성
    
    return model